In [38]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import datetime
from sklearn.preprocessing import LabelEncoder
import time
from sklearn.utils import shuffle

def readDataSets():
    dataSetFridge = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/cat/IoT_Fridge.csv')
    dataSetGarageDoor = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/cat/IoT_Garage_Door.csv')
    dataSetGPS = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/cat/IoT_GPS_Tracker.csv')
    dataSetModbus = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/cat/IoT_Modbus.csv')
    dataSetMotionLight = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/cat/IoT_Motion_Light.csv')
    dataSetThermostat = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/cat/IoT_Thermostat.csv')
    dataSetWeahter = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/cat/IoT_Weather.csv')

    dataSetFridge['temp_condition'] = dataSetFridge['temp_condition'].str.strip()
    dataSetGarageDoor['door_state'] = dataSetGarageDoor['door_state'].str.strip()
    dataSetMotionLight['light_status'] = dataSetMotionLight['light_status'].str.strip()
    dataSetRawLoad = pd.concat([dataSetFridge, dataSetGarageDoor, dataSetGPS, dataSetModbus, dataSetMotionLight, dataSetThermostat, dataSetWeahter])
    return dataSetRawLoad

In [ ]:
df = readDataSets()
print(df.shape)
print(df.info())
df.head()

a = pd.DataFrame(df['type'].value_counts())[:]
a.plot(kind='pie', subplots=True, figsize=(5, 5))
plt.title('ToN-IoT Dataset Attacks')
plt.legend(loc='left')
plt.show()

In [ ]:
newDf = df[df['type'] != 'normal']
newDf.shape

In [ ]:
newDf = newDf[newDf['type'] != 'ransomware']
newDf = newDf[newDf['type'] != 'xss']
newDf = newDf[newDf['type'] != 'scanning']
newDf.shape

In [ ]:
pd.DataFrame(newDf['type'].value_counts())[:]

In [ ]:
#matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
carrier_count = newDf['type'].value_counts()
sns.set(style="darkgrid")
sns.barplot(carrier_count.index, carrier_count.values, alpha=0.9)
plt.title('Frequency Distribution of Cat')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Cat', fontsize=12)
plt.show()

In [ ]:
dataSetRawLoad1=pd.DataFrame()
label=LabelEncoder()
for c in  newDf.columns:
    if(newDf[c].dtype=='object'):
        dataSetRawLoad1[c]=label.fit_transform(newDf[c])
    else:
        dataSetRawLoad1[c]=newDf[c]

df = dataSetRawLoad1
print(df.shape)
print(df.info())
df.head()

In [ ]:
# Load the data
categorical_features = ['door_state','sphone_signal', 'light_status','temp_condition']
#quantitative_features = ['FC1_Read_Input_Register','FC2_Read_Discrete_Value','FC3_Read_Holding_Register','FC4_Read_Coil','current_temperature',
                       #'fridge_temperature','humidity','latitude','FC4_Read_Coil','longitude',
                       # 'motion_status','pressure','temperature','thermostat_status']
quantitative_features = []   
features = categorical_features + quantitative_features

def datapreprocessingShuffle(data):
               
    # Feature scaling
    for i in quantitative_features :
            scaler = StandardScaler()
            data[i] = scaler.fit_transform(data[[i]])
            
    # Encoding categorical features    
    for i in categorical_features :
        labelencoder=LabelEncoder()
        data[i]=labelencoder.fit_transform(data[i])   
    
    data = shuffle(data).reset_index(drop=True) 
    
    Y = data.loc[:,'type']
    X = data.drop(['type'],axis=1) 
    
    return(X,Y)

In [ ]:
# Pre-processing datset
datacopy = df.copy()
X, y = datapreprocessingShuffle(datacopy) 

X = X.fillna(X.median())

In [ ]:
# Pre-processing datset
datacopy = df.copy()

datacopy = datacopy.fillna(datacopy.mean())

In [29]:
datacopy.to_csv('bal-category.csv', index=False)

In [30]:
dd = pd.read_csv(filepath_or_buffer = 'data/ToNIoT/cat/balanced/bal-category.csv')
dd.shape

In [31]:
X, y = datapreprocessingShuffle(dd) 

In [18]:
from sklearn import preprocessing
normalized_arr = preprocessing.normalize(X)
normalized_arr

In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2

# prepare target
def prepare_targets(input):
    le = LabelEncoder()
    le.fit(input)
    value = le.transform(input)
    return value

# feature selection
def select_featuresMutual(X_train, y_train, X_test):
    fs = SelectKBest(score_func=mutual_info_classif, k=4)
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs

# feature selection
def select_featuresChi(X_train, y_train, X_test):
    fs = SelectKBest(score_func=chi2, k=4)
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs

# feature selection
def select_featuresPCA(X_train, y_train, X_test, n_comp):
    fs = PCA(n_components=n_comp)
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs

In [33]:
# evaluation of a model fit using mutual information input features
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor 
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

# Spot Check Algorithms
models = []
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))
models.append(('MLP', MLPClassifier()))
models.append(('DT', DecisionTreeClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('LR', LogisticRegression()))
models.append(('SVM', svm.SVC()))
models.append(('AdaBoost', AdaBoostClassifier()))
models.append(('GradientBoosting', GradientBoostingClassifier()))
models.append(('XGB', XGBClassifier()))

for name, model in models:
    print(model)
    kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    result = next(kfold.split(X,y), None)
    x_train = X.iloc[result[0]]
    x_test = X.iloc[result[1]]
    y_train = y.iloc[result[0]]
    y_test = y.iloc[result[1]]
    model.fit(x_train, y_train)
    # evaluate the model
    y_pred = model.predict(x_test)
    print('PCA', metrics.classification_report(y_test, y_pred, target_names=['injection', 'backdoor', 'scanning', 'xss', 'password', 'ransomware', 'ddos']))

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor 
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import cross_val_predict


models = []
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))
models.append(('MLP', MLPClassifier()))
models.append(('DT', DecisionTreeClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('LR', LogisticRegression()))
models.append(('AdaBoost', AdaBoostClassifier()))
models.append(('GradientBoosting', GradientBoostingClassifier()))
models.append(('XGB', XGBClassifier()))
models.append(('SVM', svm.SVC()))
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=5, random_state=1, shuffle=True)
    #cv_results = cross_validate(model, X, y, cv=kfold, scoring=scoring)
    y_pred = cross_val_predict(model, X, y, cv=kfold)
    report = metrics.classification_report(y, y_pred)
    print(model)
    print(report)
    results.append(report)
    names.append(name)

In [ ]:
models = []
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeRegressor()))
models.append(('NB', GaussianNB()))
models.append(('MLP', MLPClassifier()))
models.append(('DT', DecisionTreeClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('LR', LogisticRegression()))
models.append(('AdaBoost', AdaBoostClassifier()))
models.append(('GradientBoosting', GradientBoostingClassifier()))
models.append(('XGB', XGBClassifier()))
models.append(('SVM', svm.SVC()))

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
          'auc-score' : make_scorer(roc_auc_score)}

# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_validate(model, X, y, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    #print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))
    print(name)
    #print(cv_results)
    print(np.mean(cv_results['test_accuracy']))
    print(np.mean(cv_results['test_precision']))
    print(np.mean(cv_results['test_recall']))
    print(np.mean(cv_results['test_f1_score']))
    print(np.mean(cv_results['test_auc-score']))